# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122373e+02     1.499354e+00
 * time: 0.08974599838256836
     1     1.093735e+01     9.874945e-01
 * time: 0.9435288906097412
     2    -1.171522e+01     1.114328e+00
 * time: 1.0625410079956055
     3    -3.384684e+01     9.477592e-01
 * time: 1.227344036102295
     4    -4.731723e+01     7.156208e-01
 * time: 1.376875877380371
     5    -5.662360e+01     2.717262e-01
 * time: 1.5210368633270264
     6    -5.954221e+01     2.019047e-01
 * time: 1.6342899799346924
     7    -6.075600e+01     6.862075e-02
 * time: 1.741919994354248
     8    -6.126781e+01     5.134828e-02
 * time: 1.8564720153808594
     9    -6.155272e+01     5.626132e-02
 * time: 1.9660019874572754
    10    -6.176416e+01     2.999491e-02
 * time: 2.0830249786376953
    11    -6.193877e+01     2.297546e-02
 * time: 2.1979339122772217
    12    -6.205427e+01     1.843503e-02
 * time: 2.3064210414886475
    13    -6.211081e+01     1.443343e-02
 * time: 2.424322843551

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671055
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522655
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336815

    total               -62.261666460518
